In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import animation
import warnings
warnings.filterwarnings('ignore')
import arviz

In [2]:
def player_list_dict_init():
    """Function returns a dictionary with the column names from the website"""
    player_list_dict = {'Runs':[],'Mins':[],'BF':[],'4s':[],'6s':[],'SR':[],'Pos':[],'Dismissal':[],'Inns':[],'Opp':[],'Ground':[],'Date':[],'#':[]}
    return player_list_dict

In [3]:
url = 'https://stats.espncricinfo.com/ci/engine/player/28114.html?class=1;template=results;type=batting;view=innings'

In [4]:
def player_df_init(url, **dictionary):
    """Function returns a dataframe of the scraped data. Takes in the url of the website and an empty dictionary"""
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    tables = soup.find_all('table',{'class':'engineTable'})
    table = tables[3]
    rows = table.find_all('tr')
    for col in rows:
        elements = col.find_all('td')
        if elements != []:
            dictionary['Runs'].append(elements[0].text.strip())
            dictionary['Mins'].append(elements[1].text.strip())
            dictionary['BF'].append(elements[2].text.strip())
            dictionary['4s'].append(elements[3].text.strip())
            dictionary['6s'].append(elements[4].text.strip())
            dictionary['SR'].append(elements[5].text.strip())
            dictionary['Pos'].append(elements[6].text.strip())
            dictionary['Dismissal'].append(elements[7].text.strip())
            dictionary['Inns'].append(elements[8].text.strip())
            dictionary['Opp'].append(elements[10].text.strip())
            dictionary['Ground'].append(elements[11].text.strip())
            dictionary['Date'].append(elements[12].text.strip())
            dictionary['#'].append(elements[13].text.strip())
    
    df = pd.DataFrame(dictionary)
    return df

In [5]:
player_dict = player_list_dict_init()
bat_df = player_df_init(url, **player_dict)
bat_df.tail()

,Runs,Mins,BF,4s,6s,SR,Pos,Dismissal,Inns,Opp,Ground,Date,#
291,29,100,73,6,0,39.72,3,bowled,3,v Australia,Sydney,3 Jan 2012,Test # 2027
292,9,40,35,1,0,25.71,3,bowled,1,v Australia,Perth,13 Jan 2012,Test # 2029
293,47,177,114,8,0,41.22,3,bowled,3,v Australia,Perth,13 Jan 2012,Test # 2029
294,1,8,9,0,0,11.11,3,bowled,2,v Australia,Adelaide,24 Jan 2012,Test # 2031
295,25,95,71,2,0,35.21,3,caught,4,v Australia,Adelaide,24 Jan 2012,Test # 2031


In [6]:
def clean(df):
    """Takes in a dataframe and returns a cleaned dataframe"""   
    # drop DNB rows
    row_dnb_drop = df[df['Runs'] == 'DNB'].index
    df_drop_dnb = df.drop(index = row_dnb_drop, axis = 0)
    
    # drop sub rows
    row_sub_drop = df_drop_dnb[df_drop_dnb['Runs'] == 'sub'].index
    df_drop_sub = df_drop_dnb.drop(index = row_sub_drop, axis = 0)
    
    # drop TDNB rows
    row_tdnb_drop = df_drop_sub[df_drop_sub['Runs'] == 'TDNB'].index
    df_drop_tdnb = df_drop_sub.drop(index = row_tdnb_drop, axis = 0)
    
    df_reset_index = df_drop_tdnb.reset_index(drop = True)
    
    # create rows and runs_clean columns
    df_reset_index['Inns'] = df_reset_index.index + 1   
    df_reset_index['Runs_clean'] = df_reset_index['Runs'].str.strip('*')
    
    # select relevant columns
    cols = ['Runs_clean','Pos','Dismissal','Inns']
    df_clean = df_reset_index[cols]
    
    return df_clean

In [7]:
bat_df_clean = clean(bat_df)
bat_df_clean.tail()

,Runs_clean,Pos,Dismissal,Inns
281,29,3,bowled,282
282,9,3,bowled,283
283,47,3,bowled,284
284,1,3,bowled,285
285,25,3,caught,286


In [8]:
def not_out(x):
    '''Function takes in an array, x, and returns numpy array, arr. It determines the number of not out's a player
    has at the i'th innings '''
    count = int(0)
    size = len(x)
    arr = np.zeros(size)
    for index, item in enumerate(x):
        if item == 'not out':
            count += 1
            arr[index] = count
        elif item == 'retired notout':
            count += 1
            arr[index] = count           
        else:
            arr[index] = count
    return arr

In [9]:
bat_df_clean['NO'] = not_out(bat_df_clean["Dismissal"]).astype('int32')
bat_df_clean.tail()

,Runs_clean,Pos,Dismissal,Inns,NO
281,29,3,bowled,282,32
282,9,3,bowled,283,32
283,47,3,bowled,284,32
284,1,3,bowled,285,32
285,25,3,caught,286,32


In [10]:
def create_cols(df):
    """Create relevant columns for the dataframe"""
    
    df['no_dismissal'] = df['Inns'] - df['NO']
    
    # assign runs_clean and pos columns to integer variables
    df['Runs_clean'] = df['Runs_clean'].astype('int32')
    df['Pos'] = df['Pos'].astype('int32')
    
    # creating a cumulative runs column
    df['Runs_cumsum'] = df['Runs_clean'].cumsum()
    df['ave'] = df['Runs_cumsum'] / df['no_dismissal']
    
    # find the position the batter batted in most
    pos = int(df['Pos'].mode()[0])
    
    cols = ['Runs_clean','no_dismissal','Runs_cumsum','Inns','ave']
    
    df_stats = df[cols]
    
    return pos, df_stats

In [11]:
pos, stats = create_cols(bat_df_clean)
stats.tail()

,Runs_clean,no_dismissal,Runs_cumsum,Inns,ave
281,29,250,13206,282,52.824000
282,9,251,13215,283,52.649402
283,47,252,13262,284,52.626984
284,1,253,13263,285,52.422925
285,25,254,13288,286,52.314961


In [12]:
def runs_var(df):
    '''Function takes in an array, x, and returns numpy array, var_arr. It determines the variance from innings 0 to innings i. '''
    size = len(df)
    var_arr = np.zeros(size)
    for i in range(size):
        var_arr[i] = np.var(df.loc[0:i,'Runs_clean'])
    return var_arr 

In [13]:
stats['var'] = runs_var(stats)
stats.tail()

,Runs_clean,no_dismissal,Runs_cumsum,Inns,ave,var
281,29,250,13206,282,52.824000,2322.169609
282,9,251,13215,283,52.649402,2319.003059
283,47,252,13262,284,52.626984,2310.837879
284,1,253,13263,285,52.422925,2310.031099
285,25,254,13288,286,52.314961,2303.570199


As we are not interested in the not out scores will drop the rows where the dismissal doesn't increase by 1 each time. Will also drop the 1st row where variance is equal to 0 this is isn't useful information. 

In [14]:
stats_drop_dup = stats.drop_duplicates("no_dismissal", keep = 'first')
mask = stats_drop_dup["var"] == 0
stats_clean = stats_drop_dup.drop(stats_drop_dup.index[mask], axis = 0)
stats_final = stats_clean.reset_index(drop = True) # reset index for ease of indentifying

In [15]:
# can now calculate the standard error
stats_final["std_err"] = (stats_final["var"] / stats_final["no_dismissal"]) ** 0.5
stats_final

,Runs_clean,no_dismissal,Runs_cumsum,Inns,ave,var,std_err
0,84,2,179,2,89.500000,30.250000,3.889087
1,8,3,187,3,62.333333,1496.222222,22.332504
2,40,4,227,4,56.750000,1215.687500,17.433355
3,24,5,251,5,50.200000,1144.160000,15.127194
4,34,6,285,6,47.500000,989.916667,12.844692
...,...,...,...,...,...,...,...
248,29,250,13206,282,52.824000,2322.169609,3.047733
249,9,251,13215,283,52.649402,2319.003059,3.039582
250,47,252,13262,284,52.626984,2310.837879,3.028199
251,1,253,13263,285,52.422925,2310.031099,3.021681


In [16]:
def bayes_calc(df):
    """Takes in the players name and role and ouputs it and other variables
    after calculating bayesian statistics using other 
    previously defined functions"""
    
    # defining the prior mean and standard deviations, and prior distribution, will also define a distribution size variable
    prior_mean = 39.15
    prior_std = 8.78
    dist_size = 100000
    prior = np.random.normal(prior_mean, prior_std, size = dist_size)
    
    # creating a likelihood dataframe l_df selecting relevant columns
    l_df = df[['no_dismissal','ave','var','std_err']]
    size = len(l_df)
    
    # creating an empty array to store the likelihood distribution 
    l_array = np.zeros(shape = (size, dist_size))
    
    # creating the posterior dictionary to store the mean and standard deviation
    pos_dict_list = {'pos_mean':[],'pos_std':[]}
    
    # creating an empty array to store the posterior distribution 
    pos_array = np.zeros(shape = (size, dist_size))
    
    for i in range(size):
        # assigning the i'th likelihood variance, innings, average, and standard error
        l_var, l_n, l_avg, l_std_err = l_df.loc[i,'var'], l_df.loc[i,'no_dismissal'], l_df.loc[i,'ave'], l_df.loc[i,'std_err']
        
        # calculating the i'th normal distribution
        l_array[i] = np.random.normal(l_avg, l_std_err, size = dist_size)
        
        # calculating and appending the posterior mean and posterior standard deviation
        pos_dict_list['pos_mean'].append(((prior_mean / (prior_std) ** 2) + ((l_n * l_avg) / l_var)) / (prior_std ** -2 + (l_n / l_var)))
        pos_dict_list['pos_std'].append((prior_std ** (-2) + (l_n / l_var)) ** (-0.5))
        
        # calculating the i'th posterior distribution
        pos_array[i] = np.random.normal(pos_dict_list['pos_mean'][i],pos_dict_list['pos_std'][i], size = dist_size)
        
    # calculating the 90% interval
    hpd = arviz.hdi(pos_array[-1], hdi_prob = 0.9)
    
    # rounding relevant statistics to 2 decimal places
    bayes_avg = round(pos_dict_list['pos_mean'][-1], 2)
    avg = round(l_df["ave"].iloc[-1], 2)
    std_err = round(pos_dict_list['pos_std'][-1], 2)
    lower_90 = round(hpd[0], 2)
    upper_90 = round(hpd[1], 2)
    
    return avg, bayes_avg, std_err, lower_90, upper_90


In [17]:
bayes_calc(stats_final)

(52.31, 50.93, 2.85, 46.19, 55.54)

In [18]:
def total(name, url):
    
    # initialise player dictionary 
    player_dict = player_list_dict_init()
    
    # initialise player dataframe
    bat_df = player_df_init(url, **player_dict)
    
    # creating a cleaned dataframe
    bat_df_clean = clean(bat_df)
    
    # creating a not out column
    bat_df_clean['NO'] = not_out(bat_df_clean["Dismissal"]).astype('int32')
    
    # finding the position batted at most by the batter, stats dataframe containing variance and standard error
    pos, stats = create_cols(bat_df_clean)
    stats['var'] = runs_var(stats)
    
    # drop rows where the no_dismissal doesn't increase by 1 
    stats_drop_dup = stats.drop_duplicates("no_dismissal", keep = 'first')
    
    # drop rows where variance is 0
    mask = stats_drop_dup["var"] == 0
    stats_clean = stats_drop_dup.drop(stats_drop_dup.index[mask], axis = 0)
    stats_final = stats_clean.reset_index(drop = True) # reset index for ease of indentifying
    
    # calculate the standard error
    stats_final["std_err"] = (stats_final["var"] / stats_final["no_dismissal"]) ** 0.5
    
    avg, bayes_avg, std_err, lower_90, upper_90 = bayes_calc(stats_final)
    
    if pos <= 3:
        role = 'Top Order'
    elif 3 < pos <= 6:
        role = 'Middle Order'
    elif 6 < pos:
        role = 'Lower Order'
    
    return name, pos, avg, bayes_avg, std_err, lower_90, upper_90, role

In [19]:
batters = pd.read_csv('batters.csv')

In [20]:
name = 'Rahul Dravid'
# url = 'https://stats.espncricinfo.com/ci/engine/player/28114.html?class=1;template=results;type=batting;view=innings'

In [21]:
name, pos, avg, bayes_avg, std_err, lower_90, upper_90, role = total(name, url)
batters = batters.append({'Name':name, 'Pos':pos ,'Avg':avg,'Bayes_Avg':bayes_avg,'std_err':std_err,\
                "90%_lower":lower_90,"90%_upper":upper_90,'label':role}, ignore_index = True)
batters.tail()
# name, pos, avg, bayes_avg, std_err, lower_90, upper_90, role

In [22]:
batters.to_csv("batters.csv", index = False) #uncomment and run when want to save as csv file

In [23]:
# label = ['Top Order', 'Middle Order','Lower Order']
# batters['label'] = pd.cut(batters['Pos'], bins = [0,3,6,7], labels = label)

In [24]:
mask_top_order = batters['label'] == 'Top Order'
mask_middle_order = batters['label'] == 'Middle Order'
mask_lower_order = batters['label'] == 'Lower Order'

In [25]:
batters[mask_top_order].sort_values('90%_lower', ascending = False)

,Name,Pos,Avg,Bayes_Avg,std_err,90%_lower,90%_upper,label
23,Kumar Sangakkara,3,57.41,54.20,3.68,48.36,60.41,Top Order
27,Rahul Dravid,3,52.31,50.94,2.83,46.18,55.49,Top Order
17,Ricky Ponting,3,51.85,50.45,2.92,45.73,55.30,Top Order
25,Kane Williamson,3,54.93,51.67,3.99,45.12,58.29,Top Order
32,Sunil Gavaskar,1,51.12,49.44,3.29,44.12,54.93,Top Order
40,Younis Khan,3,52.06,49.89,3.60,43.98,55.83,Top Order
19,Matthew Hayden,2,50.74,48.86,3.54,43.14,54.77,Top Order
39,Viv Richards,3,50.24,48.50,3.48,42.80,54.19,Top Order
5,Graeme Smith,1,48.20,46.97,3.31,41.57,52.41,Top Order
14,Marnus Labuschagne,3,53.37,49.12,4.80,41.11,56.95,Top Order


In [26]:
batters[mask_middle_order].sort_values('90%_lower', ascending = False)

,Name,Pos,Avg,Bayes_Avg,std_err,90%_lower,90%_upper,label
15,Steve Smith,4,58.62,55.12,3.72,49.00,61.23,Middle Order
56,Jacques Kallis,4,55.37,53.62,2.89,48.87,58.32,Middle Order
0,Sachin Tendulkar,4,53.80,52.30,2.80,47.75,56.98,Middle Order
21,Steve Waugh,4,51.06,49.79,2.86,45.11,54.53,Middle Order
44,Javed Miandad,4,52.57,50.33,3.58,44.33,56.15,Middle Order
31,Brian Lara,4,52.89,50.39,3.74,44.21,56.57,Middle Order
11,Joe Root,4,50.29,48.91,3.09,43.84,54.04,Middle Order
8,AB de Villiers,6,50.66,49.08,3.25,43.81,54.47,Middle Order
43,Mohammad Yousuf,4,52.29,49.83,3.80,43.56,56.00,Middle Order
46,Mike Hussey,5,51.53,49.22,3.64,43.19,55.19,Middle Order


In [27]:
batters[mask_lower_order].sort_values('90%_lower', ascending = False)

,Name,Pos,Avg,Bayes_Avg,std_err,90%_lower,90%_upper,label
52,Adam Gilchrist,7,47.61,46.14,3.66,40.13,52.11,Lower Order
53,Matt Prior,7,40.19,40.06,3.11,34.90,45.17,Lower Order
54,Brendon Mcullum,7,38.64,38.71,3.33,33.25,44.14,Lower Order
55,Imran Khan,7,37.69,37.84,2.86,33.19,42.63,Lower Order
50,Quinton de Kock,7,38.82,38.87,3.52,33.13,44.63,Lower Order
